# Parser

[Parser](https://en.wikipedia.org/wiki/Parsing) jer deo kompajlera koji na osnovu niza tokena formira [apstraktno sintaksno stablo (AST)](https://en.wikipedia.org/wiki/Abstract_syntax_tree). AST je [n-arno stablo](https://en.wikipedia.org/wiki/M-ary_tree) čiji čvorovi predstavljaju semantičke strukture sastavljene od pojedinačnih tokena. Parser će pročitati niz tokena i to je jedini put kada će se to uraditi u čitavom procesu kompajliranja. Naredne faze kompajliranja zahtevaju samo formirano stablo.

![pp-01](https://i.postimg.cc/SNmFQ6X0/pp-01.png)

**Node** class represents a base class for the AST forming. The classes that extend it match every acceptable semantic structure.


In [ ]:
class Node():
    pass

class Program(Node):
    def __init__(self, block):      
        self.block = block

class Block(Node):
    def __init__(self, decl_block, compound_statement):
        self.decl_block = decl_block
        self.compound_statement = compound_statement

class DeclBlock(Node):
    def __init__(self, nodes):      
        self.nodes = nodes

class VarBlock(Node):
    def __init__(self, nodes):      
        self.nodes = nodes

class VarDecl(Node):                   
    def __init__(self, type_, id_, assign):
        self.type_ = type_
        self.id_ = id_
        self.assign = assign

class StringDecl(Node):                   
    def __init__(self, type_, id_, size):
        self.type_ = type_
        self.id_ = id_
        self.size = size

class ArrayDecl(Node):
    def __init__(self, type_, id_, low, high, elems):
        self.type_ = type_
        self.id_ = id_
        self.low = low
        self.high = high
        self.elems = elems

class ProcDecl(Node):
    def __init__(self, id_, params, block):
        self.id_ = id_
        self.params = params
        self.block = block

class FuncDecl(Node):
    def __init__(self, type_, id_, params, block):
        self.type_ = type_
        self.id_ = id_
        self.params = params
        self.block = block

class CompoundStatement(Node):
    def __init__(self, nodes):
      self.nodes = nodes

class If(Node):
    def __init__(self, cond, true, false):
        self.cond = cond
        self.true = true
        self.false = false

class For(Node):                        # down to obraditi
    def __init__(self, init, limit, step, compound_statement):
        self.init = init
        self.limit = limit
        self.step = step
        self.compound_statement = compound_statement

class While(Node):
    def __init__(self, cond, compound_statement): 
        self.cond = cond
        self.compound_statement = compound_statement

class RepeatUntil(Node):
    def __init__(self, cond, statements):
        self.cond = cond
        self.statements = statements

class FuncCall(Node):
    def __init__(self, id_, args):
        self.id_ = id_
        self.args = args

class ProcCall(Node):
    def __init__(self, id_, args):
        self.id_ = id_
        self.args = args

class Assign(Node):
    def __init__(self, id_, expr): 
        self.id_ = id_
        self.expr = expr

class Params(Node):
    def __init__(self, params):
        self.params = params

class VarParam(Node):                   
    def __init__(self, type_, id_):
        self.type_ = type_
        self.id_ = id_

class ValueParam(Node):                   
    def __init__(self, type_, id_):
        self.type_ = type_
        self.id_ = id_

class Args(Node):
    def __init__(self, args):
        self.args = args

class ArrayElem(Node):
    def __init__(self, id_, index):
        self.id_ = id_
        self.index = index

class Elems(Node):      
    def __init__(self, elems):
        self.elems = elems

class Break(Node):
    pass

class Continue(Node):
    pass

class Exit(Node):
    pass

class Type(Node):                       # int, char etc
    def __init__(self, value):
        self.value = value

class Int(Node):                        # int value
    def __init__(self, value):
        self.value = value

class Real(Node):
    def __init__(self, value):
        self.value = value

class Boolean(Node):
    def __init__(self, value):
        self.value = value

class Char(Node):
    def __init__(self, value):
        self.value = value

class String(Node):
    def __init__(self, value):
        self.value = value

class Id(Node):
    def __init__(self, value):
        self.value = value

class BinOp(Node):
    def __init__(self, first, symbol, second):
        self.first = first
        self.symbol = symbol
        self.second = second

class UnOp(Node):
    def __init__(self, symbol, first):      
        self.symbol = symbol
        self.first = first

Modules needed for saving the object's inner state.

In [ ]:
from functools import wraps
import pickle

**Parser** class contains methods for semantic analysis of the source code. The methods use a FIFO array to form the AST (Abstract Syntax Tree) node by node. 

Method **Parse** forms the AST using [the Visitor design pattern](https://sourcemaking.com/design_patterns/visitor) by calling the method **program**.

Method **program** forms a Block node.

Method **block** forms a declaration block and a compound statement node.

Method **compound_statement** forms a node for a block of instrunctions surrounded by a begin-end statement.

Method **decl_block** forms AST declaration nodes for VAR block, procedure and function declarations.

Method **var_decl** forms nodes for variable, array, procedure and function declarations with the help of **proc_decl** and **func_decl** methods.

Method  **type_** forms a data type AST node(int,real, char, boolean).

Method **id_** forms an identificator AST node.

Method **statement** forms a node for an individual statement (one-liner or a block of statements).

Method **if_** forms nodes for condition, true statement/compound statement and an optional false statement/compound statement.

Method **while_** forms condition and compound statement nodes.

Method **for_** forms initialization, condition, step (to/downto) and compound statement nodes.

Method **while_** forms condition and statement nodes.

In [ ]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.curr = tokens.pop(0)
        self.prev = None

    def restorable(call):
        @wraps(call)
        def wrapper(self, *args, **kwargs):
            state = pickle.dumps(self.__dict__)
            result = call(self, *args, **kwargs)
            self.__dict__ = pickle.loads(state)
            return result
        return wrapper

    def eat(self, class_):
        if self.curr.class_ == class_:
            self.prev = self.curr
            self.curr = self.tokens.pop(0)
        else:
            self.die_type(class_.name, self.curr.class_.name, self.curr.lexeme, self.tokens[0].lexeme)

    def program(self):
        block = self.block()
        self.eat(Class.DOT)                                             # eof?
        return Program(block)

    def block(self):
        decl_block = self.decl_block()
        compound_statement = self.compound_statement()
        return Block(decl_block, compound_statement)

    def decl_block(self):
        decl_block = []
        while self.curr.class_ == Class.VAR or self.curr.class_ == Class.PROCEDURE or self.curr.class_ == Class.FUNCTION:
            if self.curr.class_ == Class.VAR:
                self.eat(Class.VAR)
                var_decl = []
                while self.curr.class_ == Class.ID:         
                    var_decl.extend(self.var_decl())
                    self.eat(Class.SEMICOLON)
                decl_block.append(VarBlock(var_decl))
            if self.curr.class_ == Class.PROCEDURE:
                self.eat(Class.PROCEDURE)
                proc_decl = self.proc_decl()                                     
                decl_block.append(proc_decl)
                self.eat(Class.SEMICOLON)
            if self.curr.class_ == Class.FUNCTION:                           
                self.eat(Class.FUNCTION)
                func_decl = self.func_decl()                                 
                decl_block.append(func_decl)
                self.eat(Class.SEMICOLON)
        return decl_block

    def var_decl(self):
        id_arr = []
        id_arr.append(self.id_())                                            
        while self.curr.class_ == Class.COMMA:
            self.eat(Class.COMMA)
            id_arr.append(self.id_())
        self.eat(Class.COLON)
        if self.curr.class_ == Class.TYPE:
            type_ = self.type_()
            assign = None
            if type_.value == "string":
                size = None
                if self.curr.class_ == Class.LBRACKET:
                    self.eat(Class.LBRACKET)
                    if self.curr.class_ != Class.RBRACKET:
                        size = self.expr()                                       
                    self.eat(Class.RBRACKET)
                str_decl_arr = [StringDecl(type_, id_, size) for id_ in id_arr]
                return str_decl_arr
            if self.curr.class_ == Class.EQ:                    #  assign pri deklaraciji
                if (size(id_arr) > 1):
                    self.die_deriv(self.block.__name__)
                self.eat(Class.EQ)
                expr = self.expr()  
                assign = Assign(id, expr)      
            var_decl_arr = [VarDecl(type_, id_, assign) for id_ in id_arr]
            return var_decl_arr                                           
        elif self.curr.class_ == Class.ARRAY:
            self.eat(Class.ARRAY)
            self.eat(Class.LBRACKET)
            low = Int(self.curr.lexeme)
            self.eat(Class.INT)
            self.eat(Class.RANGE)
            high = Int(self.curr.lexeme)
            self.eat(Class.INT)
            self.eat(Class.RBRACKET)
            self.eat(Class.OF)
            type_ = self.type_()
            elems = None
            if self.curr.class_ == Class.EQ:
                self.eat(Class.EQ)
                self.eat(Class.LPAREN)
                elems = self.elems()                                            
                self.eat(Class.RPAREN)
            return [ArrayDecl(type_, id_arr[0], low, high, elems)]

    def proc_decl(self):
        id_ = self.id_()
        if self.curr.class_ == Class.LPAREN:
            self.eat(Class.LPAREN)
            params = self.params()                                            
            self.eat(Class.RPAREN)
        self.eat(Class.SEMICOLON)
        block = self.block()
        return ProcDecl(id_, params, block)

    def func_decl(self):
        id_ = self.id_()
        if self.curr.class_ == Class.LPAREN:
            self.eat(Class.LPAREN)
            params = self.params()                                           
            self.eat(Class.RPAREN)
        self.eat(Class.COLON)
        type_ = self.type_()
        self.eat(Class.SEMICOLON)
        block = self.block()
        return FuncDecl(type_, id_, params, block)

    def type_(self):
        type_ = Type(self.curr.lexeme)
        self.eat(Class.TYPE)
        return type_

    def id_(self):                                                              
        is_array_elem = self.prev.class_ != Class.TYPE
        is_func_call = self.prev.class_ == Class.ASSIGN
        id_ = Id(self.curr.lexeme)
        self.eat(Class.ID)
        if self.curr.class_ == Class.LPAREN and self.is_proc_call():
            self.eat(Class.LPAREN)
            args = self.args()                                                 
            self.eat(Class.RPAREN)
            if is_func_call:
                return FuncCall(id_, args)
            return ProcCall(id_, args)
        elif self.curr.class_ == Class.LBRACKET and is_array_elem:             
            self.eat(Class.LBRACKET)
            index = self.expr()
            self.eat(Class.RBRACKET)
            id_ = ArrayElem(id_, index)
        if self.curr.class_ == Class.ASSIGN:
            self.eat(Class.ASSIGN)
            expr = self.expr()                                                 
            return Assign(id_, expr)
        else:
            return id_

    def compound_statement(self):
        self.eat(Class.BEGIN)
        nodes = [self.statement()]
        '''
        while self.curr.class_ == Class.SEMICOLON:
                self.eat(Class.SEMICOLON)
                if self.curr.class_ != Class.END:
                    nodes.append(self.statement())    
        '''
        while self.curr.class_ != Class.END:
                self.eat(Class.SEMICOLON)
                if self.curr.class_ != Class.END:
                    nodes.append(self.statement())    
        self.eat(Class.END)
        return CompoundStatement(nodes)


    def statement(self):
        '''
          if self.curr.class_ == Class.BEGIN:
              node = self.compound_statement()
        '''
        if self.curr.class_ == Class.ID:
            node = self.id_()
        elif self.curr.class_ == Class.IF:
            node = self.if_()
        elif self.curr.class_ == Class.WHILE:
            node = self.while_()
        elif self.curr.class_ == Class.FOR:
            node = self.for_()
        elif self.curr.class_ == Class.REPEAT:
            node = self.repeat_until()
        elif self.curr.class_ == Class.BREAK:
            node = self.break_()
        elif self.curr.class_ == Class.CONTINUE:
            node = self.continue_()
        elif self.curr.class_ == Class.EXIT:
            node = self.exit_()
        else:
            self.die_deriv(self.block.__name__)
        '''
        else:
            node = self.empty()
        '''
        return node

    def if_(self):
        self.eat(Class.IF)
        cond = self.logic()                                                 
        self.eat(Class.THEN)
        if self.curr.class_ == Class.BEGIN:
            true = self.compound_statement()                                
        else:
            true = self.statement()                                            
        false = None
        if self.curr.class_ == Class.ELSE:
            self.eat(Class.ELSE)
            if self.curr.class_ == Class.BEGIN:
                false = self.compound_statement()        
                flag = 1                       
            else:
                false = self.statement()        
        return If(cond, true, false)

    def while_(self):
        self.eat(Class.WHILE)
        cond = self.logic()
        self.eat(Class.DO)
        cmpd_stmt = self.compound_statement()
        return While(cond, cmpd_stmt)

    def for_(self):
        self.eat(Class.FOR)
        init = self.id_()                # tu ce se prepoznati i vratiti ASSIGN node
        if self.curr.class_ == Class.TO:
            step = Assign(init.id_, BinOp(init.id_, '+', Int(1)))
            self.eat(Class.TO)
        else:
            step = Assign(init.id_, BinOp(init.id_, '-', Int(1)))
            self.eat(Class.DOWNTO)
        limit = self.expr()                                   ### proveriti
        self.eat(Class.DO)
        cmpd_stmt = self.compound_statement()
        return For(init, limit, step, cmpd_stmt)

    def repeat_until(self):
        self.eat(Class.REPEAT)
        statements = [self.statement()]
        self.eat(Class.SEMICOLON)
        while self.curr.class_ != Class.UNTIL:
            statements.append(self.statement())
            self.eat(Class.SEMICOLON)
        self.eat(Class.UNTIL)
        cond = self.logic()
        return RepeatUntil(statements, cond)

    def params(self):                                                   
        params = []
        while self.curr.class_ != Class.RPAREN:
            if len(params) > 0:
                self.eat(Class.SEMICOLON)
            pass_by_reference = False
            if self.curr.class_ == Class.VAR:            
                self.eat(Class.VAR)                   # VAR c, d : integer
                pass_by_reference = True
            id_arr = []
            while self.curr.class_ != Class.COLON:
                if len(id_arr) > 0:
                    self.eat(Class.COMMA)
                id_arr.append(self.id_())
            self.eat(Class.COLON)
            if self.curr.class_ == Class.ARRAY:       # VAR arr: array [1..size] of integer
                self.eat(Class.ARRAY)
                low = high = None
                if (self.curr.class_ == Class.LBRACKET):      # open array parameter
                    self.eat(Class.LBRACKET)
                    low = Int(self.curr.lexeme)
                    self.eat(Class.INT)
                    self.eat(Class.RANGE)
                    high = Int(self.curr.lexeme)
                    self.eat(Class.INT)
                    self.eat(Class.RBRACKET)
                self.eat(Class.OF)
                type_ = self.type_()
                param_arr = [ArrayDecl(type_, id_arr[0], low, high, None)]
            else:
                type_ = self.type_()
                if pass_by_reference:
                  param_arr = [VarParam(type_, id_) for id_ in id_arr]
                else:
                  param_arr = [ValueParam(type_, id_) for id_ in id_arr]
            params.extend(param_arr)
        return Params(params)

    def args(self):
        args = []
        while self.curr.class_ != Class.RPAREN:
            if len(args) > 0:
                self.eat(Class.COMMA)
            args.append(self.expr())
        return Args(args);

    def elems(self):
        elems = []
        while self.curr.class_ != Class.RPAREN:
            if len(elems) > 0:
                self.eat(Class.COMMA)
            elems.append(self.expr())
        return Elems(elems)

    def exit_(self):
        self.eat(Class.EXIT)
        return Exit()

    def break_(self):
        self.eat(Class.BREAK)
        #self.eat(Class.SEMICOLON)
        return Break()

    def continue_(self):
        self.eat(Class.CONTINUE)
        #self.eat(Class.SEMICOLON)
        return Continue()

    def factor(self):
        if self.curr.class_ == Class.INT:
            value = Int(self.curr.lexeme)
            self.eat(Class.INT)
            return value
        elif self.curr.class_ == Class.REAL:
            value = REAL(self.curr.lexeme)
            self.eat(Class.REAL)
            return value
        elif self.curr.class_ == Class.BOOLEAN:
            value = Boolean(self.curr.lexeme)
            self.eat(Class.BOOLEAN)
            return value
        elif self.curr.class_ == Class.CHAR:
            value = Char(self.curr.lexeme)
            self.eat(Class.CHAR)
            return value
        elif self.curr.class_ == Class.STRING:
            value = String(self.curr.lexeme)
            self.eat(Class.STRING)
            return value
        elif self.curr.class_ == Class.ID:
            return self.id_()
        elif self.curr.class_ in [Class.MINUS, Class.NOT]:
            op = self.curr
            self.eat(self.curr.class_)
            first = None
            if self.curr.class_ == Class.LPAREN:
                self.eat(Class.LPAREN)
                first = self.logic()
                self.eat(Class.RPAREN)
            else:
                first = self.factor()
            return UnOp(op.lexeme, first)
        elif self.curr.class_ == Class.LPAREN:
            self.eat(Class.LPAREN)
            first = self.logic()
            self.eat(Class.RPAREN)
            return first
        elif self.curr.class_ == Class.SEMICOLON:
            return None
        else:
            self.die_deriv(self.factor.__name__)

    def term(self):
        first = self.factor()
        while self.curr.class_ in [Class.ASTERISK, Class.DIV, Class.FWDSLASH, Class.MOD]:
            op = self.curr.lexeme
            if self.curr.class_ == Class.ASTERISK:
                self.eat(Class.ASTERISK)
                second = self.factor()
                first = BinOp(first, op, second)
            elif self.curr.class_ == Class.DIV:
                self.eat(Class.DIV)
                second = self.factor()
                first = BinOp(first, op, second)
            elif self.curr.class_ == Class.FWDSLASH:
                self.eat(Class.FWDSLASH)
                second = self.factor()
                first = BinOp(first, op, second)
            elif self.curr.class_ == Class.MOD:
                self.eat(Class.MOD)
                second = self.factor()
                first = BinOp(first, op, second)
        return first

    def expr(self):
        first = self.term()
        while self.curr.class_ in [Class.PLUS, Class.MINUS]:
            if self.curr.class_ == Class.PLUS:
                op = self.curr.lexeme
                self.eat(Class.PLUS)
                second = self.term()
                first = BinOp(first, op, second)
            elif self.curr.class_ == Class.MINUS:
                op = self.curr.lexeme
                self.eat(Class.MINUS)
                second = self.term()
                first = BinOp(first, op, second)
        return first

    def compare(self):                                           
        first = self.expr()
        if self.curr.class_ == Class.EQ:
            op = self.curr.lexeme
            self.eat(Class.EQ)
            second = self.expr()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.NEQ:
            op = self.curr.lexeme
            self.eat(Class.NEQ)
            second = self.expr()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.LT:
            op = self.curr.lexeme
            self.eat(Class.LT)
            second = self.expr()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.GT:
            op = self.curr.lexeme
            self.eat(Class.GT)
            second = self.expr()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.LTE:
            op = self.curr.lexeme
            self.eat(Class.LTE)
            second = self.expr()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.GTE:
            op = self.curr.lexeme
            self.eat(Class.GTE)
            second = self.expr()
            return BinOp(first, op, second)
        else:
            return first

    def logic(self):
        first = self.compare()
        if self.curr.class_ == Class.AND:
            op = self.curr.lexeme
            self.eat(Class.AND)
            second = self.compare()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.OR:
            op = self.curr.lexeme
            self.eat(Class.OR)
            second = self.compare()
            return BinOp(first, op, second)
        elif self.curr.class_ == Class.XOR:
            op = self.curr.lexeme
            self.eat(Class.XOR)
            second = self.compare()
            return BinOp(first, op, second)
        else:
            return first

    @restorable
    def is_proc_call(self):
        try:
            self.eat(Class.LPAREN)
            self.args()
            self.eat(Class.RPAREN)
            #return self.curr.class_ == Class.SEMICOLON
            return True;
        except:
            return False

    def parse(self):
        return self.program()

    def die(self, text):
        raise SystemExit(text)

    def die_deriv(self, fun):
        self.die("Derivation error: {}".format(fun))

    def die_type(self, expected, found, exact, exact2):
        self.die("Expected: {}, Found: {} - {}{}".format(expected, found, exact, exact2))